In [1]:
from make_models import *
from helper_funcs import *

import numpy as np
import os 
from tqdm import tqdm
import pickle

from importlib import reload

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [3]:
df = pd.read_csv('../Data/ABIDE_df2.csv')
df = df.iloc[~pd.isna(df['bids_folder']).values]

In [4]:
data_dir = '../Assets/fc_mats_32smooth_new/'
files = [file for file in os.listdir(data_dir) if file.endswith('.npy')]
files.sort()
subs = df['bids_folder'].values
patients = df['DX_GROUP'].values==1

In [5]:
weights = [w for w in os.listdir('../Assets/tf_weights/') if w.startswith('CVAE')]
weights.sort()
weights[-5::]

['CVAE_2021-10-06 08:35:55.310173',
 'CVAE_2021-10-06 09:18:26.408246',
 'CVAE_2021-10-06 09:19:00.166747',
 'CVAE_2021-10-06 15:09:01.820789',
 'CVAE_2021-10-06 17:30:12.802507']

In [6]:
#analysis_name = 'CVAE_2021-09-15 14:03:53.826090'
analysis_name = 'CVAE_2021-10-06 08:35:55.310173'

In [7]:
data_size = (len(df),32,32,32,51)

In [8]:
import make_models;reload(make_models);from make_models import *
batch_size = 16

cvae, z_encoder, s_encoder, cvae_decoder = get_fMRI_CVAE_4D(input_shape=tuple(data_size[1::]),
                                                             latent_dim=16,
                                                             beta=1,
                                                             gamma=1,
                                                             disentangle=True,
                                                             bias=True,
                                                             batch_size = batch_size,
                                                             kernel_size = 3,
                                                             filters = 8,
                                                             intermediate_dim = 128,
                                                             nlayers = 5,
                                                             learning_rate=0.001,
                                                             opt=None)


In [9]:
cvae.load_weights(os.path.join('../Assets/tf_weights/',analysis_name,'cvae_weights'))

In [10]:
nsamples = 10
nsubs = len(df)

In [11]:
latent_dim = 16

In [12]:
bg_arr = np.zeros((nsamples,nsubs,latent_dim))
sl_arr = np.zeros((nsamples,nsubs,latent_dim))

In [13]:
for i in tqdm(range(nsamples),leave=True):
    for s in range(nsubs):
        arr = np.load(os.path.join(data_dir,subs[s]+'.npy'))
        arr = arr/2
        arr = arr[np.newaxis,:,:,:,:]

        vec_s = s_encoder.predict(arr)[2]
        vec_z = z_encoder.predict(arr)[2]

        bg_arr[i,s,:] = vec_z
        sl_arr[i,s,:] = vec_s

100%|██████████| 10/10 [13:03<00:00, 78.31s/it]


In [14]:
latent_vecs = dict()
latent_vecs['BG_ABIDE'] = bg_arr
latent_vecs['SL_ABIDE'] = sl_arr
latent_vecs['subs'] = subs

In [15]:
pickle.dump(latent_vecs,open(os.path.join('../Data/latent_vecs/','latent_vecs_'+analysis_name),'wb'))

In [16]:
#cvae_decoder.predict(vec)

In [17]:
#data_dir = '../Assets/fc_mats_32smooth_new'
#data_loader = cvae_data_loader(data_dir=data_dir, df=df, batch_size=32)